In [1]:
import gatt
import numpy as np
import time
import datetime


In [2]:
def Write_Bytes(Age ,Height, Gender, c):
    #Wait for respononse from server
    time.sleep(0.8)
    #Write bytes to initiate communication
    magicBytes = [0xac, 0x02, 0xf7, 0x00, 0x00, 0x00, 0xcc, 0xc3]
    c.write_value(magicBytes)
    #Wait for Write bytes respones
    time.sleep(1)
    #write 2nd bytes to server
    magicBytes = [0xac, 0x02, 0xfa, 0x00, 0x00, 0x00, 0xcc, 0xc6]
    c.write_value(magicBytes)
    
    #wait for Write bytes respones
    time.sleep(1)
    #Calculate offset for error checking bits
    Offset = Age + Height - 56
    #Sending Age and height with offset to server
    magicBytes = [0xac, 0x02, 0xfb, 0x01, Age, Height,  0xcc, Offset]
    c.write_value(magicBytes)
    
    time.sleep(1)
    #Calculate offset for Present date
    now = datetime.datetime.now()
    #Write present date to server(scale)
    Offset = (now.year-1799) + now.month + now.day
    magicBytes = [0xac, 0x02, 0xfd, (now.year - 2000), now.month, now.day, 0xcc, Offset]
    c.write_value(magicBytes)
    
    time.sleep(1)
    #Calculate offset for time
    now = datetime.datetime.now()
    Offset = now.hour + now.minute
    magicBytes = [0xac, 0x02, 0xfc, now.hour, now.minute, 56, 0xcc, Offset]
    c.write_value(magicBytes)
    
    time.sleep(1)
    magicBytes = [0xac, 0x02, 0xfe, 0x06, 0x00, 0x00, 0xcc, 0xd0]
    c.write_value(magicBytes)
    
    time.sleep(0.6)

def body_composition(values):
    #Weight of person shifting higher bit by 8bit position to left and or with lower bit to get 16bit value
    
    weight =  float(((values[0][12] << 8) | values[0][13]) / 10) #. kg
    Fat = float (((values[0][18] << 8 ) | values[0][19])/ 10) #.% 
    Calorie = int((values[1][5] << 8) | values[1][6] ) #. kcal
    bone_mass = float(((values[1][7] << 8 ) | values[1][8]) / 10 ) #. kg
    water = float(((values[1][9] << 8) | values[1][10]) / 10) #.% body composition
    MetabolicAge = int(values[1][11]) #In years

    print ("weight ===================>" + str(weight) +".Kg")
    print ("Fat ======================>" + str(Fat) + ".Kg")
    print ("Calorie ==================>" + str(Calorie) + "Kcal")
    print ("Bone_mass ================>" + str(bone_mass) + "Kg")
    print ("MetabolicAge =============>" + str(MetabolicAge) + "years")
    
    return {"Weight" : weight, 
            "Fat" : Fat, 
            "Calorie" : Calorie, 
            "BoneMass": bone_mass, 
            "Water" : water , 
            "MetabolicAge" : MetabolicAge }
    
    

In [6]:
class AnyDevice(gatt.Device):
    
    def define_variables(self, Age, Weight, Gender, Write):
        self.Age = Age
        self.Weight = Weight
        self.Gender = Gender
        self.Write = Write
        self.values = []
        self.ReadStatus = False
        self.Read = True
        self.count = 0
        #self.manager = manager
        
    def services_resolved(self):
        super().services_resolved()
        for s in self.services:
            if s.uuid == '0000ffb0-0000-1000-8000-00805f9b34fb': #services 0016
                for c in s.characteristics:
                    if (c.uuid == '0000ffb1-0000-1000-8000-00805f9b34fb') and (self.Write == True): #char 0017
                        Write_Bytes(self.Age, self.Weight, self.Gender, c)
                        self.Write = False
                        print(c)
                        
                        #c.enable_notifications()
                    if c.uuid == '0000ffb2-0000-1000-8000-00805f9b34fb' and self.Read == True:
                        c.read_value()
                        c.enable_notifications()
                        print(c)
                        self.Read = False
                    
    def characteristic_value_updated(self, characteristic, value):
        print("Firmware version:", value)
        
        check = value[0] + value[1]       
        if check == 0:
            self.ReadStatus = True
            
        if len(value) == 20 and self.ReadStatus == True :
            self.values.append(value)
            self.count = 1 + self.count

            if self.count == 2:
                manager.stop()
                self.count =0
            
            
        
        
    
        
    
    
        

In [7]:
manager = gatt.DeviceManager(adapter_name='hci0')
device = AnyDevice(manager=manager, mac_address='03:b3:ec:9a:97:b9')
device.define_variables(21,167 ,"male",True) # Age, Height , Gender
device.connect()
manager.run()
body_composition(device.values)



Firmware version: b'\xac\x02\xf71\x125\xcc;'
Firmware version: b'\xac\x02\xfe\t\x00\x00\xcc\xd3\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x1c\x00\x00\xcc\xe6'
Firmware version: b'\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\xac\x02\xfe\x06\xfe\x00\xcc\xce'
Firmware version: b'\xac\x02\x00\x8d\x00\x00\xce['
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xceC'
Firmware version: b'\xac\x02\x02s\x00\x00\xca?'
Firmware version: b'\xac\x02\x02s\x00\x00\xca?'
Firmware version: b'\xac\x02\xfd\x00\x00\x00\xcb\xc8'
Firmware version: b'\xac\x02\xfd\x01\x01\xe8\xcb\xb2'
Firmware v

In [8]:
body_composition(device.values)

weight ===================>62.7.Kg
Fat ======================>15.5.Kg
Calorie ==================>1496Kcal
Bone_mass ================>2.6Kg
MetabolicAge =============>16years


{'Weight': 62.7,
 'Fat': 15.5,
 'Calorie': 1496,
 'BoneMass': 2.6,
 'Water': 56.2,
 'MetabolicAge': 16}

In [ ]:
c = np.array([int(device.values[1][6])])
print(c)
# c = c.astype(float)
# c = np.sum(c, axis =0)

In [32]:
c = open('dinos.txt','r').read()
c = c.lower()
c = list(set(c))
print(c)

['z', 'p', 'x', 'k', 'u', 'b', 'c', 'v', 'e', 'w', 'd', 'r', 'o', 'y', 'q', 'f', 's', 'n', 'a', 't', 'g', 'l', '\n', 'm', 'h', 'j', 'i']


In [33]:
x = [1,3,4,5]
y = [3,4,5,6]
c = [x[i] + y [i] for i in range(len(x)) ]
print(c)

[4, 7, 9, 11]
